# Queries 

In [1]:
from bson.son import SON
from pymongo import MongoClient
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from datetime import datetime

In this notebook we perform a few queries on the MongoDB database.

## Queries on Twitch Data

In [2]:
# I have used the VM's DB
# ssh studente@10.9.13.21 -L 27017:localhost:27017

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
client.list_database_names()

['local', 'twitter', 'twitchtest', 'twitch', 'admin']

In [5]:
db = client.twitch

In [6]:
db.list_collection_names()

['games', 'system.indexes']

## /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\

# Questions 

Get all the timestamps:

In [7]:
pipeline = [{"$project":
             {'_id': 0,
              'time':'$timestamp'} }]

In [8]:
tsmps = list(db.games.aggregate(pipeline))

In [9]:
tsmps[0:15]

[{'time': '2019-06-15 19:50:59.583605'},
 {'time': '2019-06-15 19:53:59.543059'},
 {'time': '2019-06-15 19:56:59.542304'},
 {'time': '2019-06-15 19:59:59.544344'},
 {'time': '2019-06-16 16:17:09.219123'},
 {'time': '2019-06-16 16:21:13.413197'},
 {'time': '2019-06-16 16:24:13.369250'},
 {'time': '2019-06-16 16:27:13.370972'},
 {'time': '2019-06-16 16:28:43.396796'},
 {'time': '2019-06-16 16:30:13.371926'},
 {'time': '2019-06-16 16:31:10.533772'},
 {'time': '2019-06-16 16:31:18.166944'},
 {'time': '2019-06-16 16:31:43.352070'},
 {'time': '2019-06-16 16:32:05.909307'},
 {'time': '2019-06-16 16:33:13.371878'}]

In [10]:
tsmps[-100:-80]

[{'time': '2019-06-21 19:36:13.368521'},
 {'time': '2019-06-21 19:37:02.022440'},
 {'time': '2019-06-21 19:37:43.351034'},
 {'time': '2019-06-21 19:39:13.368397'},
 {'time': '2019-06-21 19:40:02.022537'},
 {'time': '2019-06-21 19:40:43.351038'},
 {'time': '2019-06-21 19:42:13.368453'},
 {'time': '2019-06-21 19:43:02.022581'},
 {'time': '2019-06-21 19:43:43.350949'},
 {'time': '2019-06-21 19:45:13.368416'},
 {'time': '2019-06-21 19:46:02.022563'},
 {'time': '2019-06-21 19:46:43.351016'},
 {'time': '2019-06-21 19:48:13.368393'},
 {'time': '2019-06-21 19:49:02.022549'},
 {'time': '2019-06-21 19:49:43.351081'},
 {'time': '2019-06-21 19:51:13.368545'},
 {'time': '2019-06-21 19:52:02.022498'},
 {'time': '2019-06-21 19:52:43.350988'},
 {'time': '2019-06-21 19:54:13.368530'},
 {'time': '2019-06-21 19:55:02.022457'}]

It looks like data has been collected more often than every 3 minutes

In [11]:
len(tsmps)

7494

In [12]:
start = tsmps[0]['time']
end = tsmps[-1]['time']
start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S.%f")
end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S.%f")

In [13]:
start, end

(datetime.datetime(2019, 6, 15, 19, 50, 59, 583605),
 datetime.datetime(2019, 6, 22, 14, 19, 2, 448362))

In [14]:
(end-start).total_seconds() / 180

3249.34924865

I dati sono stati scaricati più frequentemente che ogni 3 minuti, nonostante gli script si siano interrotti in certi momenti (magari in certi momenti c'erano più script che giravano contemporaneamente?). Non penso sia un problema, a parte che le query ci mettono più tempo a girare.

Per quanto riguarda le interruzioni e i dati mancanti, magari potremmo usare i dati che sto scaricando localmente dal 13/06.

Per adesso ci sono 3 database diversi per twitch, twitter e vgchartz. Forse conviene creare un unico database con tre collections?

## \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ 

The following is the list of games we are interested in, so that we can restrict the queries on them if we want.

In [15]:
top_games = ['Apex Legends', 'Auto Chess', 'Bio Inc. Redemption', 'Call of Duty: Advanced Warfare',
             'Call of Duty: Black Ops', 'Call of Duty: Black Ops 4',
             'Call of Duty: Black Ops II', 'Call of Duty: Black Ops III',
             'Call of Duty: Ghosts', 'Call of Duty: Modern Warfare 3',
             'Clam Man', 'Counter-Strike: Global Offensive',
             'Dead by Daylight', 'Death Stranding', 'Dota 2', 'Duck Hunt', 'FIFA 19', 'Fortnite',
             'Grand Theft Auto IV', 'Grand Theft Auto V', 'Grand Theft Auto: San Andreas', 'Hearthstone',
             'Just Chatting', 'Layers of Fear 2', 'League of Legends', 'Magic: The Gathering',
             'Mario Kart DS', 'Mario Kart Wii', 'Minecraft', 'Modern Warfare 2',
             'Music & Performing Arts', 'New Super Mario Bros.',
             'New Super Mario Bros. Wii', 'Overwatch',
             "PLAYERUNKNOWN'S BATTLEGROUNDS", 'Pokémon Gold/Silver',
             'Pokémon Red/Blue', 'Slots', 'Splitgate: Arena Warfare', 'Super Mario Bros.',
             'Super Mario World', 'Talk Shows & Podcasts', 'Tetris', 'The Elder Scrolls V: Skyrim',
             "Tom Clancy's Rainbow Six: Siege", 'Total War: Three Kingdoms',
             'Wii Play', 'Wii Sports', 'Wii Sports Resort', 'World of Warcraft']

## /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\
# Note 

There might be a couple of problematic games:
- Clam Man: for some reason was one of the top games when we initially collected the data, but is not in the recent database
- Death Stranding: not out yet,
- Just Chatting, Music & Performing Arts and Talk Shows & Podcasts are not games.

## \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ 

Average daily views for each game:

In [16]:
# MongoDB version >= 4
# pipeline = [{"$unwind": "$data"},
#             {"$match" : { "data.game_name" : { "$in" : top_games },  }},
#             { "$group": {
#                 "_id": {"name": "$data.game_name",
#                         "day": { "$dayOfMonth": {"$toDate" : "$timestamp" }},
#                         "month": { "$month": {"$toDate" : "$timestamp" }},
#                         "year": { "$year": {"$toDate" : "$timestamp" }} },
#                 "avgViewers": { "$avg": '$data.viewers'}
#             }},
#             {"$sort": {"_id": 1} }]

In [17]:
pipeline = [{"$unwind": "$data"},
            {"$match" : { "data.game_name" : { "$in" : top_games },  }},
            { "$group": {
                "_id": {"name": "$data.game_name",
                        "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                        "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                        "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                "avgViewers": { "$avg": '$data.viewers'}
            }},
            {"$sort": {"_id": 1} }]

In [ ]:
# this is VERY slow
result = list(db.games.aggregate(pipeline))

In [ ]:
result[:5]

## /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\
# Note 

In [ ]:
result_set = set([elem['_id']['name'] for elem in result])

In [ ]:
top_set = set(top_games)

In [ ]:
top_set - result_set

As mentioned above, no data was collected about Clam Man between 13/06 and 22/06, so we might want to delete it.

Also, Death Stranding has only been collected three times, on 19/06, with 1 viewer, so we might also want to substitute it with some other game.

In [ ]:
list(db.games.find({ 'data.game_name': 'Death Stranding' }, {'timestamp':1, 'data.$': 1}))

We might want to create a dataframe (to then also use it to create the visualizations):

## \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ 

In [ ]:
df = json_normalize(result)
df = df.rename(index=str, columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
df = df[['name', 'day', 'month', 'year', 'avgViewers']]

df.head(20)

## Twitter queries

In [ ]:
db = client.twitter

In [ ]:
db.list_collection_names()

Daily total retweets and likes for each game:

In [ ]:
pipeline = [{ "$group": {
                "_id": {"name": "$query",
                        "day": {"$dayOfMonth": "$date" },
                        "month": { "$month": "$date" },
                        "year": { "$year": "$date" }},
                "tot_retweets": { "$sum": "$retweets" },
                "tot_likes": { "$sum" : "$likes"},
                "tweet_count" : { "$sum": 1 }
            }},
            {"$sort": {"_id": 1} }]

In [ ]:
result = list(db.game_tweets.aggregate(pipeline))

In [ ]:
df = json_normalize(result)
df = df.rename(index=str, columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
df = df[['name', 'day', 'month', 'year', 'tot_retweets', 'tot_likes', 'tweet_count']]
df.head()

In [ ]:
df[df['name'] == 'Tetris' ]

## /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\ /|\
# Note 

Se ho capito bene i dati riguardanti il 12/06 sono stati scaricati con la versione precedente dello script di Twitter, quindi forse sono da eliminare/correggere

## \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ \\|/ 